In [1]:
import pymongo
import torch
import torchvision

from superduperdb import superduper

/Users/dodo/SuperDuperDB/superduperdb/superduperdb/misc/dataclasses.py:35: RuntimeWarning: fields may not start with an underscore, ignoring "_class_to_name"
  return add_methods(pdc.dataclass(cls, **kwargs))
/Users/dodo/SuperDuperDB/superduperdb/superduperdb/misc/dataclasses.py:35: RuntimeWarning: fields may not start with an underscore, ignoring "_lock"
  return add_methods(pdc.dataclass(cls, **kwargs))
/Users/dodo/SuperDuperDB/superduperdb/superduperdb/misc/dataclasses.py:35: RuntimeWarning: fields may not start with an underscore, ignoring "_name_to_cache"
  return add_methods(pdc.dataclass(cls, **kwargs))


In [2]:
pymongo.MongoClient().drop_database('documents')

In [3]:
class LeNet5(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(6),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(400, 120)
        self.relu = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(120, 84)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(84, num_classes)

    def preprocess(self, x):
        return torchvision.transforms.Compose([
            torchvision.transforms.Resize((32, 32)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.1307,), std=(0.3081,))]
        )(x)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

    def postprocess(self, x):
        return int(x.topk(1)[1].item())
    
model = LeNet5(10)

In [4]:
db = pymongo.MongoClient().documents

In [5]:
db = superduper(db)

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [6]:
from superduperdb.types.pillow.image import pil_image as i
from superduperdb.core.documents import Document as D
from superduperdb.queries.mongodb.queries import Collection

import random

mnist_data = list(torchvision.datasets.MNIST(root='./data', download=True))
data = [D({'img': i(x[0]), 'class': x[1]}) for x in mnist_data]
random.shuffle(data)

db.execute(
    Collection(name='mnist').insert_many(data[:-1000], encoders=[i])
)

INFO:root:Created type/pil_image/0


Stripping sub-components to references


INFO:root:found 0 uris


(<pymongo.results.InsertManyResult at 0x17030e6d0>,
 TaskWorkflow(database=<superduperdb.datalayer.base.database.BaseDatabase object at 0x16bab77c0>, G=<networkx.classes.digraph.DiGraph object at 0x17030e220>))

In [7]:
from superduperdb.queries.mongodb.queries import Collection

r = db.execute(
    Collection(name='mnist').find_one()
)

r.uri

''

In [8]:
model = superduper(model)

In [9]:
model.predict([r['img'] for r in data[:10]])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 501.15it/s]


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [10]:
# model.predict(
#     X='img',
#     db=db, 
#     select=Collection(name='mnist').find(),
#     remote=False
# )

In [11]:
from torch.optim import Adam
from torch.nn.functional import cross_entropy

from superduperdb.core.metric import Metric
from superduperdb.metrics.classification import compute_classification_metrics
from superduperdb.core.dataset import Dataset
from superduperdb.models.torch.wrapper import TorchTrainerConfiguration


job = model.fit(
    X='img',
    y='class',
    db=db,
    select=Collection(name='mnist').find(),
    configuration=TorchTrainerConfiguration(
        optimizer_cls=Adam,
        identifier='my_configuration',
        objective=cross_entropy,
        loader_kwargs={'batch_size': 10},
        max_iterations=100,
        validation_interval=10,
        compute_metrics=compute_classification_metrics,
    ),
    metrics=[Metric(identifier='acc', object=lambda x, y: x == y)],
    validation_sets=[
        Dataset(
            identifier='my_valid',
            select=Collection(name='mnist').find({'_fold': 'valid'})
        )
    ],
    remote=False
)

Stripping sub-components to references


INFO:root:Created dataset/my_valid/0
INFO:root:Created model/lenet5/0


Stripping sub-components to references


INFO:root:fold: TRAIN; iteration: 0; objective: 2.301271915435791; 
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2941/2941 [00:02<00:00, 1457.32it/s]
INFO:root:fold: VALID; iteration: 0; my_valid/acc: 0.11458687521251275; objective: 2.2991932367874406; 
INFO:root:fold: TRAIN; iteration: 1; objective: 2.372131824493408; 
INFO:root:fold: TRAIN; iteration: 2; objective: 2.3285508155822754; 
INFO:root:fold: TRAIN; iteration: 3; objective: 2.2491142749786377; 
INFO:root:fold: TRAIN; iteration: 4; objective: 2.2390012741088867; 
INFO:root:fold: TRAIN; iteration: 5; objective: 2.2737162113189697; 
INFO:root:fold: TRAIN; iteration: 6; objective: 2.265486240386963; 
INFO:root:fold: TRAIN; iteration: 7; objective: 2.1278483867645264; 
INFO:root:fold: TRAIN; iteration: 8; objective: 2.2165780067443848; 
INFO:root:fold: TRAIN; iteration: 9; objective: 2.2273449897766113; 
INFO:root:fold: TR

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2941/2941 [00:02<00:00, 1455.18it/s]
INFO:root:fold: VALID; iteration: 80; my_valid/acc: 0.8017681060863652; objective: 0.6443785047632152; 
INFO:root:fold: TRAIN; iteration: 81; objective: 0.5371196866035461; 
INFO:root:fold: TRAIN; iteration: 82; objective: 0.811449408531189; 
INFO:root:fold: TRAIN; iteration: 83; objective: 0.4538806080818176; 
INFO:root:fold: TRAIN; iteration: 84; objective: 0.8316020965576172; 
INFO:root:fold: TRAIN; iteration: 85; objective: 0.8849428296089172; 
INFO:root:fold: TRAIN; iteration: 86; objective: 0.35797756910324097; 
INFO:root:fold: TRAIN; iteration: 87; objective: 0.6734777092933655; 
INFO:root:fold: TRAIN; iteration: 88; objective: 0.7340297698974609; 
INFO:root:fold: TRAIN; iteration: 89; objective: 0.5514243245124817; 
INFO:root:fold: TRAIN; iteration: 90; objective: 0.7659941911697388; 
100%|

In [16]:
db.metadata.show_jobs()

[{'identifier': 'a76cc74f-e1ba-4476-b590-9bbd88fac5bb',
  'time': datetime.datetime(2023, 7, 3, 17, 11, 49, 392000),
  'status': 'pending'}]

In [13]:
jobs = db.add(
    Watcher(
        model=model,
        key='img',
        depends_on=model.fit(
            X='img',
            y='class',
            database=db,
            select=Collection('mnist').find(),
        )
    )
)

NameError: name 'Watcher' is not defined

In [ ]:
jobs[0].watch()

In [ ]:
from matplotlib import pyplot as plt

model = db.load('model', model.identifier)

plt.plot(model.metric_values['acc'])
plt.show()

In [ ]:
jobs[1].watch()

In [ ]:
for r in data[-1000:]:
    r['update'] = True

db.execute(Collection('docs').insert_many(data[-1000:]))

In [ ]:
db.execute(Collection('docs').find_one({'update': True}))